**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [2]:
import os
import math
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)
    
    # make a copy
    color_select = np.copy(image)
    
    gray = grayscale(color_select)
    # Gaussian Blur
    blur_gray = gaussian_blur(gray, 5)
    edges = canny(blur_gray, 50, 150)
    
    imshape = image.shape
    vertices = def_vertices(imshape)
    masked_edges = region_of_interest(edges, vertices)
    lines_edges = hough_lines(masked_edges, rho=1, theta=np.pi/180, threshold=1, min_line_len=5, max_line_gap=1)

    result = weighted_img(img=lines_edges, initial_img=image, α=1, β=100., λ=0.)
    return result
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

def getAvgLineFromListOfLines(lines):  
    #get averages of X1 and X2. 
    avgX1 = 0
    avgX2 = 0
    avgY1 = 0
    avgY2 = 0    
    
    numberOfLines = len(lines)
    if numberOfLines > 0:
        for line in lines:
            avgX1 += line.point1.x 
            avgX2 += line.point2.x
            avgY1 += line.point1.y 
            avgY2 += line.point2.y
        
        avgX1 = int(avgX1/numberOfLines)
        avgX2 = int(avgX2/numberOfLines)
        avgY1 = int(avgY1/numberOfLines)
        avgY2 = int(avgY2/numberOfLines)        
        
        avgLine = Line(Point(avgX1,avgY1),Point(avgX2,avgY2))
    else:
        print ('received empty list of Lines')
        avgLine = Line()
    return avgLine

def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    leftLines = []
    rightLines = []
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            currentLine = Line(Point(x1,y1), Point(x2,y2))
            slope = currentLine.getSlope()
            const = currentLine.getConst()
            if  slope < -0.6:
                yBottom = img.shape[0]
                xBottom = int((yBottom - const)/slope)
                yTop = int(img.shape[0]*.62)
                xTop = int((yTop - const)/slope)
                
                leftLines.append(Line(Point(xTop,yTop), Point(xBottom, yBottom)))
                #cv2.line(img, (xTop, yTop), (xBottom, yBottom), color, thickness)
                
            if  slope > 0.6:
                yBottom = img.shape[0]
                xBottom = int((yBottom - const)/slope)
                yTop = int(img.shape[0]*.62)
                xTop = int((yTop - const)/slope)
                
                rightLines.append(Line(Point(xTop,yTop), Point(xBottom, yBottom)))
                #cv2.line(img, (xTop, yTop), (xBottom, yBottom), color, thickness)         
    leftAvgLine = getAvgLineFromListOfLines(leftLines)  # get average line from a list of lines of all the left lines     
    cv2.line(img, (leftAvgLine.point1.x,leftAvgLine.point1.y), (leftAvgLine.point2.x,leftAvgLine.point2.y), color, thickness=10)

    rightAvgLine = getAvgLineFromListOfLines(rightLines)  # get average line from a list of lines of all the left lines     
    cv2.line(img, (rightAvgLine.point1.x,rightAvgLine.point1.y), (rightAvgLine.point2.x,rightAvgLine.point2.y), color, thickness=10)

    #lefAvgLine = None
    
class Point:
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y
    def __str__(self):
        return ("({0}, {1})".format(self.x, self.y))
    
class Line:
    def __init__(self, point1 = Point(), point2 = Point()):
        self.point1 = point1
        self.point2 = point2
        if point2.x - point1.x != 0:
            self.slope = (point2.y - point1.y)/(point2.x - point1.x)
        else:
            self.slope = 9999999999.0  # this is not true but lets assume for now....
        self.const = point1.y - self.slope * point1.x
    
    def calcSlope(self):
        if point2.x - point1.x != 0:
            self.slope = (point2.y - point1.y)/(point2.x - point1.x)
        else:
            self.slope = 9999999999.0  # this is not true but lets assume for now....
        return self.slope

    #def calcConst(self):

    def getSlope(self):
        return self.slope

    def getConst(self):
        return self.const

    def __str__(self):
        return ("({0} to {1}, Slope={2}, Const={3})".format(self.point1 , self.point2, self.slope, self.const))
    
leftListOfLines = []  # a global variable to store previous state of lines.  
rightListOfLines = []

def def_vertices(imshape):
    leftBottom  = Point(int(imshape[1] * 0.10), imshape[0])
    leftTop     = Point(int(imshape[1] * 0.47), int(imshape[0] * 0.60))
    rightTop    = Point(int(imshape[1] * 0.59), int(imshape[0] * 0.60))
    rightBottom = Point(int(imshape[1] * 0.95), imshape[0])
    
    return np.array([[(leftBottom.x,leftBottom.y),(leftTop.x,leftTop.y),(rightTop.x,rightTop.y),(rightBottom.x, rightBottom.y)]], dtype=np.int32)
    #print(str(leftBottom) , str(leftTop) , str(rightTop), str(rightBottom))

##### Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

In [3]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

Let's try the one with the solid white lane on the right first ...

In [4]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video white.mp4
[MoviePy] Writing video white.mp4


100%|█████████▉| 221/222 [00:02<00:00, 79.43it/s]


received empty list of Lines
received empty list of Lines
[MoviePy] Done.
[MoviePy] >>>> Video ready: white.mp4 

CPU times: user 2.22 s, sys: 748 ms, total: 2.97 s
Wall time: 3.2 s


Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [5]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

**At this point, if you were successful you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform.  Modify your draw_lines function accordingly and try re-running your pipeline.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [6]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video yellow.mp4
[MoviePy] Writing video yellow.mp4


 24%|██▍       | 164/682 [00:02<00:08, 62.62it/s]

received empty list of Lines


 26%|██▌       | 178/682 [00:02<00:08, 60.65it/s]

received empty list of Lines
received empty list of Lines
received empty list of Lines


...

received empty list of Lines


 98%|█████████▊| 668/682 [00:09<00:00, 70.91it/s]

received empty list of Lines


100%|█████████▉| 681/682 [00:09<00:00, 69.97it/s]


received empty list of Lines
[MoviePy] Done.
[MoviePy] >>>> Video ready: yellow.mp4 

CPU times: user 7.49 s, sys: 2.27 s, total: 9.76 s
Wall time: 10.2 s


In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Reflections

Congratulations on finding the lane lines!  As the final step in this project, we would like you to share your thoughts on your lane finding pipeline... specifically, how could you imagine making your algorithm better / more robust?  Where will your current algorithm be likely to fail?

Please add your thoughts below,  and if you're up for making your pipeline more robust, be sure to scroll down and check out the optional challenge video below!


## Submission

If you're satisfied with your video outputs it's time to submit!  Submit this ipython notebook for review.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video extra.mp4
[MoviePy] Writing video extra.mp4


 40%|████      | 101/251 [00:02<00:04, 36.93it/s]

received empty list of Lines
received empty list of Lines
received empty list of Lines


 45%|████▌     | 113/251 [00:02<00:03, 36.09it/s]

received empty list of Lines


 50%|████▉     | 125/251 [00:03<00:03, 35.78it/s]

received empty list of Lines


 53%|█████▎    | 133/251 [00:03<00:03, 32.17it/s]

received empty list of Lines


100%|██████████| 251/251 [00:07<00:00, 32.98it/s]


In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))